## 1. Setup Environment

In [1]:
# John Knott
# 4.7.2025
# Homework 8

!pip install spacy
!pip install pandas
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ----------- ---------------------------- 3.7/12.8 MB 21.8 MB/s eta 0:00:01
     ------------------ --------------------- 6.0/12.8 MB 16.1 MB/s eta 0:00:01
     ------------------------ --------------- 7.9/12.8 MB 13.9 MB/s eta 0:00:01
     ------------------------------- ------- 10.2/12.8 MB 12.8 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 12.2 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 11.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# import libraries
import spacy
import pandas as pd

In [3]:
# load Spacy model
nlp = spacy.load("en_core_web_sm")

## 2. Load the Dataset

In [4]:
# load stocks-1.tsv
df = pd.read_csv("stocks-1.tsv", sep="\t")
# examine column names and data structure
print(df.head())

  Symbol            CompanyName                        Industry MarketCap
0      A   Agilent Technologies  Life Sciences Tools & Services    53.65B
1     AA                  Alcoa                 Metals & Mining     9.25B
2    AAC       Ares Acquisition                 Shell Companies     1.22B
3   AACG  ATA Creativity Global   Diversified Consumer Services    90.35M
4   AADI        Aadi Bioscience                 Pharmaceuticals   104.85M


## 3. Extract Data for Patterns

In [5]:


# Load file with relative path
df = pd.read_csv("stocks-1.tsv", sep="\t")

# extract unique stock symbols and names
symbols = df['Symbol'].dropna().unique()
company_names = df['CompanyName'].dropna().unique()

# create patterns using for loops

symbol_patterns = []
for symbol in symbols:
    pattern = {"label": "STOCK", "pattern": symbol}
    symbol_patterns.append(pattern)


company_patterns = []
for name in company_names:
    pattern = {"label": "COMPANY", "pattern": name}
    company_patterns.append(pattern)

# combine all patterns
all_patterns = symbol_patterns + company_patterns




## 4. Create an Entity Ruler


In [6]:
#import entity ruler
from spacy.pipeline import EntityRuler
# Load spaCy model and add EntityRuler 
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler", before="ner")
ruler.add_patterns(all_patterns)



## 5. Test the EntityRuler

In [7]:
#load text
text1 = "Helmerich & Payne (HP) saw its stock rise by 1.5%, fueled by optimistic forecasts in the Energy Equipment & Services sector. In contrast, Check-Cap (CHEK) faced a decline of 2.3% following its announcement of increased costs related to supply chain disruptions.Meanwhile, Vallon Pharmaceuticals (VLON) gained 0.8% after strong quarterly earnings, outperforming its peers in the Biotechnology space. Sequans Communications (SQNS) also recorded a modest increase of 0.5%, reflecting investors' confidence in its ability to navigate challenges in the Semiconductors & Semiconductor Equipment industry."
# Process the text using spacy model
doc1 = nlp(text1)
# print results
for ent in doc1.ents:
    print(ent.text, ent.label_)

Helmerich & Payne COMPANY
HP STOCK
1.5% PERCENT
the Energy Equipment & Services ORG
Check-Cap COMPANY
CHEK STOCK
2.3% PERCENT
Vallon Pharmaceuticals COMPANY
VLON STOCK
0.8% PERCENT
quarterly DATE
Biotechnology ORG
Sequans Communications COMPANY
SQNS STOCK
0.5% PERCENT
Semiconductors & Semiconductor Equipment ORG


Notes:
Gets Organizations wrong. 
Quarterly is not a date.
Gets percent, stocks, and companies right

In [8]:
text2 = "Aemetis (AMTX) saw its stock rise by 1.5%, fueled by optimistic forecasts in the Oil, Gas & Consumable Fuels sector. In contrast, Ferro Corporation (FOE) faced a decline of 2.3% following its announcement of increased costs related to supply chain disruptions.Meanwhile, RingCentral (RNG) gained 0.8% after strong quarterly earnings, outperforming its peers in the Software space. ACI Worldwide (ACIW) also recorded a modest increase of 0.5%, reflecting investors' confidence in its ability to navigate challenges in the Software industry."
doc2 = nlp(text2)
for ent in doc2.ents:
    print(ent.text, ent.label_)

Aemetis COMPANY
AMTX STOCK
1.5% PERCENT
the Oil, Gas & Consumable Fuels ORG
Ferro Corporation COMPANY
FOE STOCK
2.3% PERCENT
RingCentral COMPANY
RNG STOCK
0.8% PERCENT
quarterly DATE
Software ORG
ACI Worldwide COMPANY
ACIW STOCK
0.5% PERCENT
Software ORG


Notes:
Still gets ORG's wrong for example gets the Oil, Gas & Consumable Fuels wrong and labels it as an ORG

In [9]:
text3 = "On a mixed trading day, Par Pacific Holdings (PARR) saw its stock rise by 1.5%, fueled by optimistic forecasts in the Oil, Gas & Consumable Fuels sector. In contrast, Nano Dimension (NNDM) faced a decline of 2.3% following its announcement of increased costs related to supply chain disruptions.Meanwhile, Beyond Meat (BYND) gained 0.8% after strong quarterly earnings, outperforming its peers in the Food Products space. Apollo Investment (AINV) also recorded a modest increase of 0.5%, reflecting investors' confidence in its ability to navigate challenges in the Capital Markets industry."
doc3 = nlp(text3)
for ent in doc3.ents:
    print(ent.text, ent.label_)

Par Pacific Holdings COMPANY
PARR STOCK
1.5% PERCENT
the Oil, Gas & Consumable Fuels ORG
Nano Dimension COMPANY
NNDM STOCK
2.3% PERCENT
Beyond Meat COMPANY
BYND STOCK
0.8% PERCENT
quarterly DATE
Food Products ORG
Apollo Investment COMPANY
AINV STOCK
0.5% PERCENT
Capital Markets ORG


Notes: 
gets ORG's wrong. Everything else is correct.

# Takeaways:

In order to better be analyze these text files I need to be able to differentiate industries from Organizations. I would essentially do the same thing I did in part 3 but take into account the Industry column when adding patterns to my entity ruler.